## 流量を管理するモジュール  

本モジュールで計算できる流量は2018/10/9現在以下の3つである。

- $q_s$ 熱流（固体熱伝導）[J/m2s]
- $j_v$ 水蒸気流 [kg/m2s]
- $j_l$ 液水流 [kg/m2s]
  

流量の与え方は大まかに  

- 第二種境界条件；一定値を与える方法  
- 第三種境界条件；両端のセル情報から流量を計算する方法  

の2種類がある。しかしながら外壁表面などでは第二種と第三種境界条件を組み合わせた条件などが存在する。（日射や降雨が当たる場合など）  
そのため流量の管理モジュールは以下の三種類を有する必要がある。

- 第二種境界条件
- 第三種境界条件（流量計算）  
- 第二種＋第三種境界条件  

ここでは第二種境界条件のクラスと第三種境界条件のクラスを分けることでそれぞれのクラスを作成した。


### 第二種境界条件  

第二種境界条件のクラスではそれぞれ熱流・水蒸気流・液水流を保持するだけのクラスとする。


In [1]:
class NeumannBoundaryCondition():
    
    def __init__( self ):
        
        self.qs = 0.0
        self.jv = 0.0
        self.jl = 0.0
        
    # qs
    
    def set_qs(self, qs):
        self.qs = qs

    # jv
    
    def set_jv(self, jv):
        self.jv = jv
    
    # jl
    
    def set_jl(self, jl):
        self.jl = jl
    

### 第三種境界条件

第三種境界条件として2つのセル情報からセル間の流量を計算するクラス。  
流量の具体的な計算方法については下記の"流量の計算方法"に定義することとした。

流量を計算するにあたり、材料間の質点の距離、物性値の平均値、セル間の重力に対する応答方向などを取る必要がある。
特に質点間の距離については質点の設定という工程を省略するため、
ここでは全てのセルの質点を中央に取ることにより質点間の距離を簡単に計算できる方式を採用した。  

そのため空気と接する材料境界では熱及び水分の移動は、  

空気から材料表面のへの熱・水分流　＋　材料表面からの材料の質点までの熱・水分流

という形で表した。なお材料表面から空気への水分流は水蒸気のみになるとし、液相水分流は生じないものとした。


In [2]:
class RobinBoundaryCondition():
    
    def __init__( self, cell_mns, cell_pls, nx ):
        
        self.cell_mns = cell_mns
        self.cell_pls = cell_pls
        self.nx = nx
        
    # dx2 質点間の距離
    def cal_dx2(self):
        return self.cell_mns.dx / 2.0 + self.cell_pls.dx / 2.0
    dx2 = property(cal_dx2)
    
    ################################
    # 物性値の平均化
    # lam 熱伝導率
    def cal_lam(self):
        return sum_resistance( val_mns=self.cell_mns.lam, val_pls=self.cell_pls.lam, len_mns=self.cell_mns.dx, len_pls=self.cell_pls.dx )
    lam = property(cal_lam)
        
    # dp 湿気伝導率
    def cal_dp(self):
        return sum_resistance( val_mns=self.cell_mns.dp, val_pls=self.cell_pls.dp, len_mns=self.cell_mns.dx, len_pls=self.cell_pls.dx )
    dp = property(cal_dp)
        
    # ldml 液水の水分伝導率
    def cal_ldml(self):
        return sum_resistance( val_mns=self.cell_mns.ldml, val_pls=self.cell_pls.ldml, len_mns=self.cell_mns.dx, len_pls=self.cell_pls.dx )
    ldml = property(cal_ldml)
        
    ################################
    # 熱流計算
    
    def cal_heat_conduction( self ):
        
        # 両方のセルがともに多孔質材料である場合
        if hasattr(self.cell_mns, 'material') and hasattr(self.cell_pls, 'material'):
            dtemp  = self.cell_pls.material.temp - self.cell_mns.material.temp
            return heat_conduction( self.lam, dtemp, self.dx2 )
        
        # 片方（pls側）のセルに多孔質材料がセットされてない場合
        elif hasattr(self.cell_pls, 'material'):
            K = transmittance( self.cell_mns.alpha, self.cell_pls.lam, self.cell_pls.dx )
            return heat_transfer( K, self.cell_mns.air.temp, self.cell_pls.material.temp )
        
        # 片方（mns側）のセルに多孔質材料がセットされてない場合
        elif hasattr(self.cell_mns, 'material'):
            K = transmittance( self.cell_pls.alpha, self.cell_mns.lam, self.cell_mns.dx )
            return - heat_transfer( K, self.cell_pls.air.temp, self.cell_mns.material.temp )
        
        else:
            raise NameError('Cellクラスの設定が適切ではありません。')
    
    qs = property(cal_heat_conduction)
    
    # 水蒸気流計算（水蒸気圧差）
    
    def cal_vapour_permeance( self ):
        
        # 両方のセルがともに多孔質材料である場合
        if hasattr(self.cell_mns, 'material') and hasattr(self.cell_pls, 'material'):
            dpv  = self.cell_pls.vapour.pv - self.cell_mns.vapour.pv
            return vapour_permeance_pressure( self.dp, dpv, self.dx2 )
        
        # 片方（pls側）のセルに多孔質材料がセットされてない場合
        elif hasattr(self.cell_pls, 'material'):
            Kp = transmittance( self.cell_mns.aldm, self.cell_pls.dp, self.cell_pls.dx )
            return vapour_transfer_pressure( Kp, self.cell_mns.air.pv, self.cell_pls.vapour.pv )
        
        # 片方（mns側）のセルに多孔質材料がセットされてない場合
        elif hasattr(self.cell_mns, 'material'):
            Kp = transmittance( self.cell_pls.aldm, self.cell_mns.dp, self.cell_mns.dx )
            return - vapour_transfer_pressure( Kp, self.cell_pls.air.pv, self.cell_mns.vapour.pv )
        
        else:
            raise NameError('Cellクラスの設定が適切ではありません。')
    
    jv = property(cal_vapour_permeance)
    
    # 液水流計算（水分化学ポテンシャル差）
    
    def cal_liquid_conduction( self ):
    
        # 両方のセルがともに多孔質材料である場合
        if hasattr(self.cell_mns, 'material') and hasattr(self.cell_pls, 'material'):
            dmiu  = self.cell_pls.water.miu - self.cell_mns.water.miu
            return liquid_conduction_potential( self.ldml, dmiu, self.dx2, self.nx )

        else:
            return 0.0
    
    jl = property(cal_liquid_conduction)
    

### 第二種 + 第三種境界条件  

熱伝達＋日射というような形の境界条件の場合こちらを用いる。（等価外気温の形を用いても良いが）  

基本構造は第二種と第三種のクラスの組み合わせである。

In [3]:
class CombinationBoundaryCondition():
    
    def __init__( self, cell_mns, cell_pls, nx ):
        self.neumann = NeumannBoundaryCondition()
        self.robin   = RobinBoundaryCondition(cell_mns, cell_pls, nx)
        
    # Neumann qs
    def set_qs(self, qs):
        self.neumann.set_qs(qs) 

    # Neumann jv
    def set_jv(self, jv):
        self.neumann.set_jv(jv)
    
    # Neumann jl
    def set_jl(self, jl):
        self.neumann.set_jl(jl)
    
    ################################
    # 熱流計算
    
    def sum_heat_conduction( self ):
        return self.neumann.qs + self.robin.qs
    
    qs = property(sum_heat_conduction)
    
    # 水蒸気流計算（水蒸気圧差）
    
    def sum_vapour_permeance( self ):
        return self.neumann.jv + self.robin.jv
    
    jv = property(sum_vapour_permeance)
    
    # 液水流計算（水分化学ポテンシャル差）
    
    def sum_liquid_conduction( self ):
        return self.neumann.jl + self.robin.jl
        
    jl = property(sum_liquid_conduction)
    

#### 重力加速度

In [4]:
def grav():
    return 9.806650

# 流量の計算方法

### 物性値の平均化   
セル間の物性値を計算するにあたり2つのセルの物性値を平均化する必要がある。  
この際、平均化の方法についてはいくつかやり方が存在するがここでは以下の2つの方法について示す。

#### 抵抗値として重ね合わせる  
セル間の流量が一定の場合、その間の物性値は各セルの物性値の抵抗値の和として表すことができる。したがって、  

熱伝導抵抗：  $R[m^2K/W] =\frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} }  $  

同様に水分に関しても抵抗値の和として計算することで物性値を平均化することができる。  
ここで熱伝導率・水分伝導率を合わせて$\lambda$と表記すると、平均の伝導率は以下のように表すことができる。  

平均の伝導率：  $\lambda =\frac{ dx_{l+1} + dx_{l} }{\frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} } } $  


In [5]:
def sum_resistance( val_mns, val_pls, len_mns, len_pls ):
    return (len_mns + len_pls) / ( len_mns / val_mns + len_pls / val_pls )

In [6]:
# 使用例
sum_resistance( val_mns = 10.0, val_pls = 1.0, len_mns = 0.1, len_pls = 0.1 )

1.8181818181818183

#### 補足：片方が空気の場合  
材料が空気と接する場合、空気から材料表面への熱移動は熱伝達及び放射により生じ、材料中から材料表面までは熱伝導によって生じる。  
このような場合、材料表面に質点を置くことが一般的であるが、質点の位置をCellクラスに入れることは非常に手間がかかる。  
そのためここでは空気から材料中への熱流が定常状態であるとし、熱抵抗の和として計算を行うこととする。   
空気および材料の熱貫流率は以下の式で表される。  

熱貫流率：$K =\frac{ 1 }{ \frac{ 1 }{ \alpha } + \frac{ dx_{l} }{ 2\lambda_{l} } } $  


In [7]:
def transmittance( alpha, lam, dx ):
    return ( 1.0 ) / ( 1.0 / alpha + dx / ( 2.0 * lam ) )

#### 調和平均を取る  
一方でセルの長さに応じた物性値の調和平均を取るという考え方もある。
この平均化の仕方は伝導率の差が小さい時は問題ないが、差が大きくなると抵抗値の考え方と大きくずれてくる。

物性値の調和平均：　　$\lambda_{ave}=\frac{\lambda_{l+1} dx_{l+1} + \lambda_{l} dx_{l} }{ dx_{l+1} + dx_{l} }$  


In [8]:
def mean_average( val_mns, val_pls, len_mns, len_pls ):
    return ( val_mns * len_mns + val_pls * len_pls ) / ( len_mns + len_pls )

In [9]:
# 使用例
mean_average( val_mns = 10.0, val_pls = 1.0, len_mns = 0.1, len_pls = 0.1 )

5.5

### 熱伝導（フーリエの法則）  
定義：$\dot q = -\lambda\nabla・T$  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\lambda$：熱伝導率[W/mK]  
$T$：絶対温度[K]  

In [10]:
def heat_conduction( lam, dtemp, dx2 ):
    return - lam * dtemp / dx2

#### 差分化その1：$\dot q = -\lambda_{ave.}\frac{T_{l+1}-T_{l}}{dx_{l+1}+dx_{l}}$  
$\lambda_{ave.}$：平均の熱伝導率  
$l, l+1$：ある点のセル・隣接する点のセル  
$dx$：セルの質点からセル境界までの距離

In [11]:
def heat_conduction_v1( lamL1, lamL, tempL1, tempL, dxL1, dxL ):
    lamAve = mean_average( val_mns = lamL, val_pls = lamL1, len_mns = dxL, len_pls = dxL1 )
    return - lamAve * ( tempL1 - tempL ) / ( dxL1 + dxL )

#### 差分化その2：$\dot q = -K(T_{l+1}-T_{l})$  
$K$：熱貫流率[W/K]  
一般的にはこちらを採用。異種材料間の熱の移動は熱抵抗値の和として計算。

In [12]:
def heat_conduction_v2( lamL1, lamL, tempL1, tempL, dxL1, dxL ):
    K = transmittance( val_mns = lamL, val_pls = lamL1, len_mns = dxL, len_pls = dxL1 )
    return - K * ( tempL1 - tempL )

### 熱伝達・熱貫流  

定義：$\dot q = \alpha(T_{air} - T_{wall})$  
空気に接する壁表面には空気の熱伝達層が存在すると考えた場合、もしくは質点間の熱貫流率として計算した場合の熱流。  
ここでは便宜上、空気と壁表面の熱伝達を想定し、空気から壁表面への熱流を正とした。

$\dot q$：単位時間当たりの熱流量の密度[W/m2]  ]
$\alpha$：熱伝達率・熱貫流率[W/m2K]  
$T_{air}$：空気の絶対温度[K]  
$T_{wall}$：壁表面の絶対温度[K]  

In [13]:
def heat_transfer( alpha, temp_air, temp_wall ):
    return alpha * ( temp_air - temp_wall )

## 水蒸気移動  
元原理：Fickの拡散法則

### 水蒸気移動（水蒸気圧勾配・固体内）  
定義：$J_v = -\lambda^{'}_{p}\nabla・P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\lambda^{'}_{p}$：水蒸気圧勾配に関する気相水分伝導率[kg/msPa]  
$P_v$：水蒸気圧[Pa]  

In [14]:
def vapour_permeance_pressure( dp, dpv, dx2 ):
    return - dp * dpv /dx2

### 湿気伝達・湿気貫流  
定義：$J_v = \alpha^{'}_{m}(P_{v_{air}} - P_{v_{wall}})$  
空気に接する壁表面には空気の湿気伝達層が存在すると考えた場合の水分流（水蒸気流）、もしくは質点間の湿気貫流率として計算した場合の水分流。  
ここでは便宜上、空気と壁表面の水分伝達を想定し、空気から壁表面への水分流を正とした。

$J_v$：単位時間当たりの水分流量[kg/m2s]  
$\alpha^{'}_{m}$：湿気伝達率[kg/m2s(Pa)]  
$P_{v_{air}}$：空気の水蒸気圧[Pa]  
$P_{v_{wall}}$：壁表面の水蒸気圧[Pa]  

In [15]:
def vapour_transfer_pressure( alpha, pv_air, pv_wall ):
    return alpha * ( pv_air - pv_wall )

## 液水移動  
原理：ダルシー則

### 液水移動（水分化学ポテンシャル勾配・固体内）  
定義：$J_l = -\lambda^{'}_{\mu_l}(\nabla・\mu - n_x g)$    

$J_l$：液相水分流量[kg/m2s]  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  
$\mu$：液水の化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答（重力加速度に対して水平なら1、垂直なら0）

In [16]:
def liquid_conduction_potential( ldml, dmiu, dx2, nx ):
    return - ldml * ( dmiu / dx2 - nx * grav() )